In [ ]:
source("shared.R")

In [ ]:
data("PlantGrowth")
head(PlantGrowth)

In [ ]:
boxplot(weight ~ group, data = PlantGrowth, main = "PlantGrowth data",
        ylab = "Dried weight of plants", col = "lightgray",
        notch = FALSE, varwidth = TRUE)

In [ ]:
#?effects
#?anova
anova(lm(weight ~ group, data = PlantGrowth))

In [ ]:
lmod = lm(weight ~ group, data=PlantGrowth)
summary(lmod)

In [ ]:
str_model = "model {
    for (i in 1:length(y)) { 
        y[i] ~ dnorm(mu[grp[i]], prec)
    }
    
    for (j in 1:3) {
        mu[j] ~ dnorm(0, 1./1.e6)
    }

    # here 5 is the sample size, 1 is the variance
    prec ~ dgamma(5/2.0, 5*1/2.0) 

    sig = sqrt(1./prec)    
} "

dat = c()
var_names = c("mu", "sig")
inits = function() {
    inits = list("mu" = rnorm(3, 0, 100), "prec" = rgamma(1, 1., 1.))
}
data = list(y=PlantGrowth$weight, grp=as.numeric(PlantGrowth$group))

In [ ]:
mod = jags_model_compile(str_model_ = str_model, data_ = data, inits_ = inits, n_chains_ = 3, burnout = 1e5)
sim = jags_model_sample(mod_ = mod, n_iter_ = 1e4, var_names_ = var_names)
mod_diag = jags_model_diag(samples_ = sim$samples)

In [ ]:
pm_params = colMeans(sim$samples_cmb)
summary(sim$samples)

In [ ]:
HPDinterval(sim$samples_cmb[,3] - sim$samples_cmb[,1])

In [ ]:
HPDinterval(sim$samples_cmb, prob = 0.9)

In [ ]:
mod_cm = lm(weight ~ -1 + group, data=PlantGrowth)
summary(mod_cm)

In [ ]:
HPDinterval(sim$samples_cmb, prob = 0.9)

## Fitting model with varied variance for each group

In [ ]:
model_varied_var = "model {
    for (i in 1:length(y)) { 
        y[i] ~ dnorm(mu[grp[i]], prec[grp[i]])
    }    
    for (j in 1:3) {
        mu[j] ~ dnorm(0, 1./1.e6)
        prec[j] ~ dgamma(5/2.0, 5*1/2.0) 
        sig[j] = sqrt(1./prec[j])
    }    
} "


var_names = c("mu", "sig")
inits = function() {
    inits = list("mu" = rnorm(3, 0, 100), "prec" = rgamma(3, 1., 1.))
}
data = list(y=PlantGrowth$weight, grp=as.numeric(PlantGrowth$group))

In [ ]:
jmod_varied_var = jags_model_compile(model_varied_var, data, inits)
sim2 = jags_model_sample(jmod_varied_var, n_iter_ = 1e4, var_names_ = var_names)

In [ ]:
dic.samples(mod, n.iter=1e5) - dic.samples(jmod_varied_var, n.iter=1e5)

In [ ]:
d1 = dic.samples(model = jmod_varied_var, n.iter = 1e4)

In [ ]:
dics = list("m1" = dic.samples(model = jmod_varied_var, n.iter = 1e4),  "m2" = dic.samples(model = mod, n.iter = 1e4))

In [ ]:
a = dic.samples(model = jmod_varied_var, n.iter = 1e4)

In [ ]:
summary(sim$samples_cmb)

In [ ]:
y_hat = pm_params[1:3][data$grp]
resid = data$y - y_hat
plot(resid)

In [ ]:
plot(y_hat, resid)